# Data Processing Module

## 0. Downloading Dataset

In this project, we use `Vietnamese car price` dataset, which contains information of 30.7k used car on a Vietnamese online market at the beginning of 2023

In [114]:
# Download Dataset using gdown
# https://drive.google.com/file/d/1uhhpMRqBVTTG5AYdddhYjvj26lVYnExL/view?usp=sharing
!pip install gdown==4.6.0 -q
!gdown 1uhhpMRqBVTTG5AYdddhYjvj26lVYnExL

Downloading...
From: https://drive.google.com/uc?id=1uhhpMRqBVTTG5AYdddhYjvj26lVYnExL
To: /content/vietnamese_car_price.zip
100% 12.2M/12.2M [00:00<00:00, 96.5MB/s]


In [115]:
# Unzip dataset
!unzip -qq -a vietnamese_car_price.zip

replace car_detail.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## 1. Import Libraries

In [143]:
# Import necessary libraries
import numpy as np # Import NumPy for numerical computations and data manipulation

import pandas as pd  # Import Pandas for data manipulation and analysis
pd.set_option('display.max_rows', 1000) # Set Pandas options to display a maximum of 1000 rows

import re # Import re for working with text and regular expressions

# Import warnings to ignore error
import warnings
warnings.filterwarnings("ignore")

## 2. Loading Dataset

In [331]:
# Loading dataset
car_details = pd.read_csv('car_detail_en.csv')
# Rename columns
car_details.rename(columns={'price. price': 'price'}, inplace=True)

# Check for dupication
item0 = car_details.shape[0]  # Stores the initial number of rows in the DataFrame
car_details = car_details.drop_duplicates()  # Removes duplicate rows from the DataFrame
item1 = car_details.shape[0]  # Stores the number of rows after removing duplicates
print(f"There are {item0-item1} duplicates found in the dataset")  # Prints the number of duplicates that were removed

There are 0 duplicates found in the dataset


In [332]:
car_details.head().T

,0,1,2,3,4
ad_id,17042,53794,73954,74150,87573
origin,Domestic assembly,Imported,Domestic assembly,Imported,Domestic assembly
condition,New car,New car,New car,New car,New car
car_model,Truck,SUV,Crossover,SUV,Crossover
mileage,0 Km,0 Km,0 Km,0 Km,0 Km
exterior_color,White,Black,Silver,White,Silver
interior_color,gray,Black,Brown,Black,gray
num_of_doors,2-door,5-door,5-door,5-door,5-door
seating_capacity,2-seat,7-seat,8-seat,5-seat,8-seat
engine,Petrol\t1.0 L,Petrol\t3.4 L,Petrol\t2.0 L,Petrol\t1.8 L,Petrol\t2.0 L


## 3. Data Preprocessing

### Convert `price` to numerical

In [333]:
# Before processing
car_details["price"].head()

0              249 Million
1    4 Billion 286 Million
2              885 Million
3              754 Million
4              850 Million
Name: price, dtype: object

In [334]:
 # Function to convert price to numerical
def price_to_numeric(price):
    try:
        if "Billion" in price and not "Million" in price:
            return float(price.replace("Billion", "").replace(" ", ""))*1000
        elif "Million" in price and not "Billion" in price:
            return float(price.replace("Million", "").replace(" ", ""))
        else:
            return float(price.replace("Billion", "").replace("Million", "").replace(" ", ""))
    except:
        return None

# Convert price to numerical
car_details['price'] = car_details['price'].apply(price_to_numeric) # Convert price to numerical
car_details['log10_price'] = np.log10(car_details['price']) # Standardization
car_details = car_details[~car_details['price'].isnull()] # Drop missing data

In [335]:
# After Processing
car_details["price"].head()

0     249.0
1    4286.0
2     885.0
3     754.0
4     850.0
Name: price, dtype: float64

### Convert `mileage` to numerical

In [336]:
# Before Processing
car_details["mileage"].head()

0    0 Km
1    0 Km
2    0 Km
3    0 Km
4    0 Km
Name: mileage, dtype: object

In [337]:
# Function to convert mileage to numerical
def mileage_to_numeric(mileage):
    try:
        return float(mileage.rstrip(' Km').replace(" ", '').replace(',',''))
    except:
        return None

# Convert mileage to numerical
car_details['mileage'] = car_details['mileage'].apply(mileage_to_numeric) # Convert mileage to numerical
car_details['log10_mileage'] = np.log10(car_details['mileage']+1) # Standardization (+1 to avoid Zero Devision Error)

In [338]:
# After Processing
car_details["mileage"].head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: mileage, dtype: float64

### Get `fuel_consumption`

In [339]:
# Before Processing
car_details["fuel_consumption"].head()

0        L/100Km
1    10\tL/100Km
2        L/100Km
3        L/100Km
4        L/100Km
Name: fuel_consumption, dtype: object

In [340]:
# Define a function to get fuel consumptions
def get_fuel_consumption(fuel_consumption):
    result = str(fuel_consumption).replace("L/100Km", "").rstrip("\t").replace(',','').strip() # Remove non-numeric charater
    pattern = r'[A-Za-z]'
    result = re.sub(pattern,'', result)  # Remove non-numeric charaters
    if result:
        return float(result)
    else:
        return 'None'

# Get fuel consumptions
car_details['fuel_consumption'] = car_details['fuel_consumption'].apply(get_fuel_consumption)

In [341]:
# Before Processing
car_details["fuel_consumption"].head()

0    None
1    10.0
2    None
3    None
4    None
Name: fuel_consumption, dtype: object

### Get `engine` information

In [342]:
# Before Processing
car_details["engine"].head()

0    Petrol\t1.0 L
1    Petrol\t3.4 L
2    Petrol\t2.0 L
3    Petrol\t1.8 L
4    Petrol\t2.0 L
Name: engine, dtype: object

In [343]:
# Get fuel-type
car_details['fuel_type'] = car_details['engine'].apply(lambda engine: str(engine).split('\t')[0])

# Get engine size
def get_engine_type(engine):
    result = str(engine).split('\t')[-1]
    if "L" in result:
        return result
    else:
        return 'None'

car_details['engine_size'] = car_details['engine'].apply(get_engine_type)

In [344]:
# After Processing
car_details[["engine", "fuel_type", "engine_size"]].head()

,engine,fuel_type,engine_size
0,Petrol\t1.0 L,Petrol,1.0 L
1,Petrol\t3.4 L,Petrol,3.4 L
2,Petrol\t2.0 L,Petrol,2.0 L
3,Petrol\t1.8 L,Petrol,1.8 L
4,Petrol\t2.0 L,Petrol,2.0 L


### Combine `brand` and `grade`

In [345]:
# Before Processing
car_details[["brand", "grade"]].head()

,brand,grade
0,Suzuki,Super Carry Truck
1,Toyota,Land Cruiser
2,Toyota,Innova
3,Toyota,Corolla Cross
4,Toyota,Innova


In [346]:
# Combine brand and grade
car_details['brand_grade'] = car_details['brand'] + ' ' + car_details['grade']

In [347]:
# After Processing
car_details[["brand", "grade", "brand_grade"]].head()

,brand,grade,brand_grade
0,Suzuki,Super Carry Truck,Suzuki Super Carry Truck
1,Toyota,Land Cruiser,Toyota Land Cruiser
2,Toyota,Innova,Toyota Innova
3,Toyota,Corolla Cross,Toyota Corolla Cross
4,Toyota,Innova,Toyota Innova


### Get Car Name

In [348]:
# Before Processing
car_details[["brand", "grade", "brand_grade", "car_name"]].head()

,brand,grade,brand_grade,car_name
0,Suzuki,Super Carry Truck,Suzuki Super Carry Truck,Suzuki Super Carry Truck 1.0 MT 2022
1,Toyota,Land Cruiser,Toyota Land Cruiser,Toyota Land Cruiser 3.5 V6 2022
2,Toyota,Innova,Toyota Innova,Toyota Innova G 2.0 AT 2023 car
3,Toyota,Corolla Cross,Toyota Corolla Cross,Toyota Corolla Cross 1.8G 2023
4,Toyota,Innova,Toyota Innova,Toyota Innova G 2.0 AT 2022 car


In [349]:
# Define a function to get car model
def get_car_model(data_row):
    substring = data_row['brand_grade'] + ' '
    if substring in data_row['car_name']:
        data_row['car_name'] = data_row['car_name'].replace(substring, '')
        return data_row
    else:
        return data_row

# Get car model
car_details = car_details.apply(get_car_model, axis=1)

In [359]:
# Before Processing
car_details[["brand", "grade", "brand_grade", "car_name"]].head()

,brand,grade,brand_grade,car_name
0,Suzuki,Super Carry Truck,Suzuki Super Carry Truck,1.0 MT 2022
1,Toyota,Land Cruiser,Toyota Land Cruiser,3.5 V6 2022
2,Toyota,Innova,Toyota Innova,G 2.0 AT 2023 car
3,Toyota,Corolla Cross,Toyota Corolla Cross,1.8G 2023
4,Toyota,Innova,Toyota Innova,G 2.0 AT 2022 car


### Get number of doors

In [351]:
# Before Processing
car_details["num_of_doors"].head()

0    2-door
1    5-door
2    5-door
3    5-door
4    5-door
Name: num_of_doors, dtype: object

In [352]:
# Define a function to get number of doors
def get_num_of_doors(num_of_doors):
    result = int(str(num_of_doors).split('-')[0])
    return result

# Get number of doors
car_details['num_of_doors'] = car_details['num_of_doors'].apply(get_num_of_doors)

# Remove abnormal data (number of door normally is not greater than 6)
item0 = car_details.shape[0] # Stores the initial number of rows in the DataFrame
car_details = car_details[car_details['num_of_doors']<=7.0] # Remove abnormal data form the Dataframe
item1 = car_details.shape[0] # Stores the number of rows after removing abnormal data
print(f"There are {item0-item1} abnormal data of num_of_doors found in the dataset")  # Prints the number of abnormal data that were removed

There are 8 abnormal data of num_of_doors found in the dataset


In [353]:
car_details["num_of_doors"].head()

0    2
1    5
2    5
3    5
4    5
Name: num_of_doors, dtype: int64

### Get seating capacity

In [354]:
# Before Processing
car_details["seating_capacity"].head()

0    2-seat
1    7-seat
2    8-seat
3    5-seat
4    8-seat
Name: seating_capacity, dtype: object

In [355]:
# Define a function to get seating_capacity
def get_seating_capacity(seating_capacity):
    result = int(str(seating_capacity).split('-')[0])
    return result

# Get seating_capacity
car_details['seating_capacity'] = car_details['seating_capacity'].apply(get_seating_capacity) # Get seating_capacity

# Remove abnormal data (number of door can not lower than 0)
item0 = car_details.shape[0]  # Stores the initial number of rows in the DataFrame
car_details = car_details[car_details['seating_capacity']>0].reset_index() # Remove abnormal data form the Dataframe
item1 = car_details.shape[0]  # Stores the number of rows after removing abnormal data
print(f"There are {item0-item1} abnormal data of seating_capacity found in the dataset")  # Prints the number of abnormal data that were removed

There are 55 abnormal data of seating_capacity found in the dataset


In [356]:
# After Processing
car_details["seating_capacity"].head()

0    2
1    7
2    8
3    5
4    8
Name: seating_capacity, dtype: int64

### Get seller address

In [357]:
# Load sellers dataset
sellers = pd.read_csv("seller.csv")

# Rename Columns
sellers.rename(columns = {"Mã tin": 'ad_id',
                          "Địa chỉ": "seller_address"}, inplace=True)

# Select only specific columns of interest
sellers = sellers[['ad_id', 'seller_address']]

# Get city name from address
def get_city_name(address):
  # Remove duplicate space in string
  pattern = ' +' # More than 1 space continuously
  address = re.sub(pattern, ' ', address)
  # Check if 2-3 last word are city name
  if ' '.join(address.strip(' ').split(' ')[-2:]) == 'Thiên Huế':
    return ' '.join(address.strip(' ').split(' ')[-3:])
  else:
    return ' '.join(address.strip(' ').split(' ')[-2:])

sellers['seller_address'] = sellers['seller_address'].apply(get_city_name)

# Merge car_details with sellers
car_details = pd.merge(car_details, sellers, on="ad_id", how="left")

In [358]:
# After Processing
car_details["seller_address"].head()

0    Hà Nội
1    Hà Nội
2    TP HCM
3    TP HCM
4    Hà Nội
Name: seller_address, dtype: object

## Export Result to CSV file

In [363]:
# Select only specific columns of interest
selected_cols = ['price', 'log10_price', 'brand_grade', 'car_name', 'car_model',
                 'origin', 'condition', 'mileage', 'log10_mileage','exterior_color',
                 'interior_color', 'num_of_doors', 'seating_capacity','fuel_type',
                 'engine_size', 'transmission','drive_type', 'fuel_consumption',
                 'year_of_manufacture', 'seller_address']
car_details = car_details[selected_cols]

# Export Report to csv
car_details.to_csv("car_details_preprocessed.csv",index=False)